In [1]:
# モデルのテストと、sagemakerのデプロイを行うファイル
%ls
%pwd

Dockerfile
Faster_R_CNN_ver_sagemaker_moduled.ipynb
README.md
archived/
build_and_push.sh
custombdd/
docs/
models/
modules/
sagemaker_entry_point.py
scripts/
tutorial/


'/Users/tamae/gitdir/maskrcnn'

In [ ]:
!nvidia-smi

In [2]:
from modules.infer import testShowInferenceImage
from modules.train import train
from modules.MLparamaters import getMLParamaters


paramaters are
 dict_items([('model_path', '/Users/tamae/gitdir/maskrcnn/models/model.pt'), ('test_path', '/Users/tamae/gitdir/maskrcnn/custombdd/test'), ('bdd_img', '/Users/tamae/gitdir/maskrcnn/custombdd/img'), ('bdd_xml', '/Users/tamae/gitdir/maskrcnn/custombdd/xml'), ('dataset_class', ['person', 'vehicle', 'c_vehicle2', 'c_vehicle3', 'tank_lorry', 'truck', 'c_vehicle', 'c_vehicle4']), ('colors', ((0, 0, 0), (255, 0, 0), (0, 255, 0), (0, 0, 255), (100, 255, 100), (100, 100, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255))), ('epochs', 20), ('batch_size', 4), ('scale', 1024), ('device', device(type='cpu')), ('useS3', False)])


In [ ]:

# print(torch.__version__)
# print(torchvision.__version__)
# print(torchtext.__version__)
# print(torchaudio.__version__)
# 1.13.1
# 0.14.1
# 0.14.1
# 0.13.1

In [ ]:

# ! echo checkpoint1

# ここ、pip installは初回のみでいいかも
# !pip install -r requirements.txt
# %pwd


In [3]:
#学習済みモデルで推論する場合
paramaters = getMLParamaters()

#train()

testShowInferenceImage(paramaters.model_path)

START testShowInferenceImage
type: <class 'torchvision.models.detection.faster_rcnn.FasterRCNN'>


In [ ]:
# !tar zcvf model.tar.gz models/model.pt
import sagemaker

input_s3_uri = sagemaker.session.Session().upload_data(path='./custombdd/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='custombdd')
# s3://sagemaker-ap-northeast-1-958305726855/custombdd/

sagemaker.__version__


In [ ]:
# sagamakerでエンドポイントデプロイ関連

from sagemaker import get_execution_role
role = get_execution_role()
print(role)


from sagemaker.pytorch.model import PyTorchModel

s3_path="s3://mlearning-bucket/sageMakerDeployTest/model/model.tar.gz"

image="958305726855.dkr.ecr.ap-northeast-1.amazonaws.com/test:latest"


pytorch_model = sagemaker.estimator.Estimator(model_data=s3_path,
                            #source_dir='.',によって、dockerで起動されるインスタンスに最新のスクリプトを送っている
                            source_dir='.',
                            entry_point='sagemaker_entry_point.py',
                            image_uri=image,
                             role=role,
                            train_instance_count=1,
                            train_instance_type='ml.g4dn.xlarge'
                                             )


pytorch_model.fit(input_s3_uri)

In [ ]:
# デプロイ
deploy_params = {
    'instance_type'         : 'ml.g4dn.xlarge',
    'instance_count' : 1
}
predictor = pytorch_model.deploy(**deploy_params)

In [ ]:
# デプロイしたものの画像表示テスト

import matplotlib.pyplot as plt
import numpy as np
import cv2

filename = 'testInf.jpg'

# インライン表示
%matplotlib inline

# Read image into memory
payload = None
with open(filename, 'rb') as f:
    payload = f.read()

#画像の表示
frombuff = np.frombuffer(payload, dtype='uint8') # byteをnumpyにしている(1次元)

input_data = cv2.imdecode(frombuff, cv2.IMREAD_UNCHANGED) # numpyを画像の形式にdecodeしている。

bytearray_data = bytearray(payload)
plt.imshow(input_data) 
plt.show()

from sagemaker.serializers import IdentitySerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = IdentitySerializer()
predictor.deserializer = JSONDeserializer()
results = predictor.predict(bytearray_data, initial_args={'ContentType': 'application/x-image'})

print(results)

In [ ]:
# ログ書き出しテスト

import json
import time
import datetime
import boto3

client = boto3.client('logs')

def get_log_stream_token(log_group_name, log_stream_name):
    # ログイベントがログストリームに存在する場合、書き込むを行うにはログストリームのトークンが必要なのでそれを取得する
    response = client.describe_log_streams(
        logGroupName=log_group_name,
        logStreamNamePrefix=log_stream_name,
    )
    log_streams = response['logStreams']
    log_stream = log_streams[0]
    log_stream_token = log_stream['uploadSequenceToken']
    return log_stream_token

def put_custom_log_event(log_group_name, log_stream_name, log_stream_token):
    # ログストリームのトークンを元にログイベントを書き込む
    response = client.put_log_events(
        logGroupName=log_group_name,
        logStreamName=log_stream_name,
        logEvents=[
            {
                'timestamp': int(time.time()) * 1000,
                'message': 'testほげ'
            },
        ],
        sequenceToken=log_stream_token
      )

def main():
    log_group_name = '/aws/sagemaker/Endpoints/sagemakerinftest3'
    log_stream_name = 'AllTraffic/i-03f1ee612f0e1aa0a'
    log_stream_token= get_log_stream_token(log_group_name, log_stream_name)
    put_custom_log_event(log_group_name, log_stream_name, log_stream_token)

main()